In [ ]:
from pathlib import Path
import utils as utl
import pandas as pd
import seaborn as sns
import matplotlib.pylab as plt
import numpy as np

path_name = './flywire_ids_630.xlsx'
path_comp = './2023_03_23_completeness_630_final.csv'

# sheet names of the xls file to include
sheets_pair = [ # pair of neurons (left+right per row)
    # 'stop',
    # 'walk',
    'walk_outputs',
    # 'SEZ',
    ]
sheets_single = [ # single neurons (one per row)
    # 'sugar', 
    # 'ovidn', 
    # 'bitter', 
    # 'water', 
    # 'jo_e', 
    # 'jo_f', 
    # 'eye_bristle', 
    ]

name2flyid = utl.create_name_dict(path_name, path_comp, sheets_pair, sheets_single)

In [ ]:
# load raw data, save rates
p_out = Path('./2freq_plots/')

fs = [ *range(20, 141, 20) ]

def save_comb(combination, fs, root, p_out):
    
    ps = []
    for f1 in fs:
        for f2 in fs:
            n = f'{combination}_{f1}_{f2}.parquet'
            ps.append(root / n)

    df_spkt = utl.load_exps(ps)
    df_rate, _ = utl.get_rate(df_spkt, duration=1)
    df_rate.to_parquet(p_out / f'rate_{combination}.parquet')


# combination = 'sugarr+p9'
# root = Path('./results/coac2/sens-walk/')
# save_comb(combination, fs, root, p_out)

# combination = 'sugarr+p9-fg'
# root = Path('./results/coac2slnc/sens-walk/')
# save_comb(combination, fs, root, p_out)

# combination = 'sugarr+p9-bb'
# root = Path('./results/coac2slnc/sens-walk/')
# save_comb(combination, fs, root, p_out)

# combination = 'sugarr+p9-bb-fg'
# root = Path('./results/coac2slnc/sens-walk/')
# save_comb(combination, fs, root, p_out)

# combination = 'sugarr+bpn'
# root = Path('./results/coac2/sens-walk/')
# save_comb(combination, fs, root, p_out)

# combination = 'sugarr+bpn-fg'
# root = Path('./results/coac2slnc/sens-walk/')
# save_comb(combination, fs, root, p_out)

# combination = 'sugarr+bpn-bb'
# root = Path('./results/coac2slnc/sens-walk/')
# save_comb(combination, fs, root, p_out)

# combination = 'sugarr+bpn-bb-fg'
# root = Path('./results/coac2slnc/sens-walk/')
# save_comb(combination, fs, root, p_out)


In [ ]:
tcc = [
    ('P9', 'P9-oDN1_r', 'sugarr+p9', 'sugarr+p9-fg'),
    ('P9', 'P9-oDN1_l', 'sugarr+p9', 'sugarr+p9-fg'),
    ('P9', 'P9-oDN1_r', 'sugarr+p9', 'sugarr+p9-bb'),
    ('P9', 'P9-oDN1_l', 'sugarr+p9', 'sugarr+p9-bb'),
    ('P9', 'P9-oDN1_r', 'sugarr+p9', 'sugarr+p9-bb-fg'),
    ('P9', 'P9-oDN1_l', 'sugarr+p9', 'sugarr+p9-bb-fg'),
    ('BPN', 'BPN_SEZ_DN2_r', 'sugarr+bpn', 'sugarr+bpn-fg'),
    ('BPN', 'BPN_SEZ_DN2_l', 'sugarr+bpn', 'sugarr+bpn-fg'),
    ('BPN', 'BPN_SEZ_DN2_r', 'sugarr+bpn', 'sugarr+bpn-bb'),
    ('BPN', 'BPN_SEZ_DN2_l', 'sugarr+bpn', 'sugarr+bpn-bb'),
    ('BPN', 'BPN_SEZ_DN2_r', 'sugarr+bpn', 'sugarr+bpn-bb-fg'),
    ('BPN', 'BPN_SEZ_DN2_l', 'sugarr+bpn', 'sugarr+bpn-bb-fg'),
]

df_diff_min, df_diff_max = 0, 0

for name, target1, combination1, combination2 in tcc:

    # construct table for target neuron
    target2 = target1

    df_rate = pd.read_parquet(p_out / f'rate_{combination1}.parquet')
    df_rate = utl.rename_index(df_rate, name2flyid)

    ds = df_rate.loc[target1, :]
    ids = [i for i in ds.index if not ('200' in i or '180' in i or '160' in i) ]
    ds = ds.loc[ids]

    fs1 = [ int(i.split('_')[1]) for i in ds.index ]
    fs2 = [ int(i.split('_')[2]) for i in ds.index ]

    df = pd.pivot_table(
        data=pd.DataFrame(data={
            'r': ds.values,
            'f1': fs1, # sugar activation
            'f2': fs2
    }), values='r', index='f2', columns='f1') # sugar becomes columns

    df = df.sort_index(ascending=False)

    df1 = df
    df_rate = pd.read_parquet(p_out / f'rate_{combination2}.parquet')
    df_rate = utl.rename_index(df_rate, name2flyid)

    ds = df_rate.loc[target2, :]
    ids = [i for i in ds.index if not ('200' in i or '180' in i or '160' in i) ]
    ds = ds.loc[ids]
    fs1 = [ int(i.split('_')[1]) for i in ds.index ]
    fs2 = [ int(i.split('_')[2]) for i in ds.index ]

    df = pd.pivot_table(
        data=pd.DataFrame(data={
            'r': ds.values,
            'f1': fs1, # sugar activation
            'f2': fs2
    }), values='r', index='f2', columns='f1') # sugar becomes columns

    df = df.sort_index(ascending=False)

    df2 = df

    # plot

    fig, axarr = plt.subplots(ncols=3, figsize=(12, 3))

    # vmax = max(df1.values.max(), df2.values.max()) # set to None to not match scale
    norm = df1.values.max()
    vmax = np.max([df1.values.max() / norm, df2.values.max() / norm ])
    cmap = 'viridis'
    print(f'{combination2} {target1}: {df1.values.max():1.2f}, {df2.values.max():1.2f}, {vmax:1.2f}')
    ax = axarr[0]
    
    sns.heatmap(ax=ax, data=df1 / norm, cmap=cmap, vmin=0, vmax=1.05, square=True, cbar_kws={'label': f'rel {target1} firing rate [Hz]'})
    ax.set_ylabel(f'{name} activation [Hz]')

    ax = axarr[1]
    sns.heatmap(ax=ax, data=df2 / norm, cmap=cmap, vmin=0, vmax=1.05, square=True, cbar_kws={'label': f'rel {target2} firing rate [Hz]'})
    ax.set_ylabel(f'{name} activation [Hz]')

    ax = axarr[2]
    # df_diff = ((df2 - df1) / df1).replace([np.inf, -np.inf, np.nan], 0)
    df_diff = (df2 - df1) / norm
    sns.heatmap(ax=ax, data=df_diff, cmap='coolwarm', center=0, vmin=-.06, vmax=0.53, square=True, cbar_kws={'label': f'delta {target2} firing rate [Hz]'})
    ax.set_ylabel(f'{name} activation [Hz]')
    df_diff_min = np.min([df_diff_min, df_diff.values.min()])
    df_diff_max = np.max([df_diff_max, df_diff.values.max()])

    for ax in axarr:
        ax.set_yticklabels(ax.get_yticklabels(), rotation=0)
        ax.set_xlabel('sugar activation [Hz]')

    fig.tight_layout()
    fig.savefig(p_out / f'norm/{combination2}_{target1}.eps')
    fig.savefig(p_out / f'norm/{combination2}_{target1}.png')
    # break
print()
print(df_diff_min, df_diff_max)

In [ ]:
# construct table for target neuron
target1 = 'BPN_SEZ_DN2_r'
target2 = target1

combination1 = 'sugarr+bpn'
combination2 = 'sugarr+bpn-fg'

df_rate = pd.read_parquet(p_out / f'rate_{combination1}.parquet')
df_rate = utl.rename_index(df_rate, name2flyid)

ds = df_rate.loc[target1, :]
ids = [i for i in ds.index if not ('200' in i or '180' in i or '160' in i) ]
ds = ds.loc[ids]

fs1 = [ int(i.split('_')[1]) for i in ds.index ]
fs2 = [ int(i.split('_')[2]) for i in ds.index ]

df = pd.pivot_table(
    data=pd.DataFrame(data={
        'r': ds.values,
        'f1': fs1, # sugar activation
        'f2': fs2
}), values='r', index='f2', columns='f1') # sugar becomes columns

df = df.sort_index(ascending=False)

df1 = df
df_rate = pd.read_parquet(p_out / f'rate_{combination2}.parquet')
df_rate = utl.rename_index(df_rate, name2flyid)

ds = df_rate.loc[target2, :]
ids = [i for i in ds.index if not ('200' in i or '180' in i or '160' in i) ]
ds = ds.loc[ids]
fs1 = [ int(i.split('_')[1]) for i in ds.index ]
fs2 = [ int(i.split('_')[2]) for i in ds.index ]

df = pd.pivot_table(
    data=pd.DataFrame(data={
        'r': ds.values,
        'f1': fs1, # sugar activation
        'f2': fs2
}), values='r', index='f2', columns='f1') # sugar becomes columns

df = df.sort_index(ascending=False)

df2 = df

In [ ]:
# plot

fig, axarr = plt.subplots(ncols=3, figsize=(12, 3))

vmax = max(df1.values.max(), df2.values.max()) # set to None to not match scale
cmap = 'viridis'

ax = axarr[0]
sns.heatmap(ax=ax, data=df1, cmap=cmap, vmin=0, vmax=vmax, square=True, cbar_kws={'label': f'{target1} firing rate [Hz]'})
ax.set_ylabel('P9 activation [Hz]')

ax = axarr[1]
sns.heatmap(ax=ax, data=df2, cmap=cmap, vmin=0, vmax=vmax, square=True, cbar_kws={'label': f'{target2} firing rate [Hz]'})
ax.set_ylabel('BPN activation [Hz]')

ax = axarr[2]
sns.heatmap(ax=ax, data=df2 - df1, cmap='coolwarm', center=0, square=True, cbar_kws={'label': f'delta {target2} firing rate [Hz]'})
ax.set_ylabel('BPN activation [Hz]')

for ax in axarr:
    ax.set_yticklabels(ax.get_yticklabels(), rotation=0)
    ax.set_xlabel('sugar activation [Hz]')

fig.tight_layout()
# fig.savefig('heatmap_2freq_new.svg')